In [1]:
!pip install shap

  Using cached shap-0.46.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
Using cached shap-0.46.0-cp312-cp312-macosx_11_0_arm64.whl (455 kB)
Using cached slicer-0.0.8-py3-none-any.whl (15 kB)


In [ ]:
import shap
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 모델 파일 경로
model_path = '/Users/jiminsim/Desktop/project/HR/IBM_model_1.keras'

# 모델 로드
model = tf.keras.models.load_model(model_path)

# 데이터 로드
df = pd.read_csv('/Users/jiminsim/Desktop/project/HR/전처리 데이터/IBM_2.csv')

# 파생 변수 추가
df['Long_Distance'] = df['DistanceFromHome'].apply(lambda x: 1 if x > 10 else 0)

# 종속 변수와 독립 변수 분리
X = df.drop('Attrition', axis=1)  
y = df['Attrition']

# 데이터 정규화
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SHAP Explainer 생성
explainer = shap.KernelExplainer(model.predict, X_train)  # 전체 데이터를 사용

# SHAP 값 계산
shap_values = explainer.shap_values(X_test)  # 전체 테스트 데이터를 사용

# shap_values 차원 확인
print("shap_values 차원:", np.array(shap_values).shape)

# shap_values의 마지막 차원을 제거 (len(X_test), 30) 형태로 변환
shap_values = np.squeeze(shap_values, axis=-1)
print("차원 변환 후 shap_values 차원:", shap_values.shape)

# SHAP 시각화 (변수 중요도)
X_test2 = pd.DataFrame(X_test, columns=df.drop('Attrition', axis=1).columns)  # 전체 테스트 데이터 사용

# 모든 피처를 표시하도록 설정
shap.summary_plot(shap_values, X_test2, feature_names=X_test2.columns, max_display=len(X_test2.columns))
